In [1]:
#default_exp core

In [2]:
# !pip install nbdev

# restaurants_timeseries

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import os
import pandas as pd
DATAPATH = 'data'

DATA_FILES = {
    "visits": os.path.join(DATAPATH, "air_visit_data.csv"),
    "store_info": os.path.join(DATAPATH, "air_store_info.csv"),
    "reservations": os.path.join(DATAPATH, "air_reserve.csv"),
    "date_info": os.path.join(DATAPATH, "date_info.csv")
}

def read_data():
    return dict([(name, pd.read_csv(path)) for name, path in DATA_FILES.items()])

#export
data = read_data()

In [ ]:
for name, df in data.items():
    print(name)
    display(df.head())

In [ ]:
from nbdev.export import *
notebook2script()